In [1]:
%%capture
from ovcDreams import ovcS_dreams as OVC,  regime_status as REGIMSTAT, positive_dreams
from hivstat_func import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")

# HIVSTAT

In [2]:
OVC.test_results.value_counts()

0,       3676
0,,2,     390
2,        321
1,          1
Name: test_results, dtype: int64

In [3]:
REGIMSTAT[REGIMSTAT.id_patient.isin(OVC.id_patient)]
positive_dreams[positive_dreams.id_patient.isin(REGIMSTAT.id_patient)]

,patient_code,id_patient,start_date,regime,all_start_date,all_end_date


,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,number_of_condoms_sensibilize,number_condoms_reception_in_the_interval,number_test_date_in_the_interval,...,ps_1519,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ4


# HIVSTAT without HIV screening

In [4]:
%%capture
HIVSTAT = OVC
HIVSTAT['test not indicated'] = HIVSTAT.test_results.map(test_indication)
HIVSTAT['positive'] = HIVSTAT.test_results.map(test_evaluation)
HIVSTAT['negative'] = HIVSTAT.test_results.map(test_evaluation)
HIVSTAT['not_on_arv'] = HIVSTAT.test_results.map(arv)
HIVSTAT['on_arv'] = 'no'
HIVSTAT['No HIV reported (Status Unknown)'] = 0


In [5]:
positive = HIVSTAT[HIVSTAT.positive == 'positive']
not_indicated = HIVSTAT[HIVSTAT['test not indicated'] == 'ni']
negative = HIVSTAT[HIVSTAT.negative == 'negative']
on_arv = HIVSTAT[(HIVSTAT.positive=='positive')&(HIVSTAT.on_arv!='no')]
not_on_arv = HIVSTAT[(HIVSTAT.positive=='positive')&(HIVSTAT.on_arv=='no')]

In [6]:
positive = positive.pivot_table(index="commune", values='id_patient', columns='test_results',aggfunc='count',fill_value=0)
positive.rename(columns={'1,':'positive'},inplace=True)
positive.reset_index(inplace=True)


negative = negative.pivot_table(index="commune", values='id_patient', columns='negative',aggfunc='count',fill_value=0)
negative.reset_index(inplace=True)

on_arv = on_arv.pivot_table(index="commune", values='id_patient', columns='on_arv',aggfunc='count',fill_value=0)
on_arv.reset_index(inplace=True)



not_on_arv = not_on_arv.pivot_table(index="commune", values='id_patient', columns='on_arv',aggfunc='count',fill_value=0)
not_on_arv.rename(columns={'no':'not_on_arv'},inplace=True)
not_on_arv.reset_index(inplace=True)



not_indicated = not_indicated.pivot_table(index="commune", values='id_patient', columns='test not indicated',aggfunc='count',fill_value=0)
not_indicated.rename(columns={'ni':'test not indicated'},inplace=True)
not_indicated.reset_index(inplace=True)

In [7]:

index = ['commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No_HIV_reported_Status_Unknown']

   
   
#Calling the function
DATIMHIVSTAT= ovc_hivstat_children(index,positive,negative,on_arv,not_on_arv,not_indicated)
DATIMHIVSTAT.fillna(0,inplace=True)


In [8]:
DATIMHIVSTAT= DATIMHIVSTAT.astype(int, errors='ignore')

In [9]:
if DATIMHIVSTAT[['positive','negative','test not indicated','No_HIV_reported_Status_Unknown']].sum(axis=1).sum() == OVC.id_patient.count(): 
    print("DATIM IS VALID")
    DATIMHIVSTAT

DATIM IS VALID


,commune,positive,on_arv,not_on_arv,negative,test not indicated,No_HIV_reported_Status_Unknown
0,Cap-Haïtien,0,0,0,58,378,0
1,Carrefour,0,0,0,18,232,0
2,Delmas,0,0,0,6,134,0
3,Desdunes,0,0,0,31,133,0
4,Dessalines,0,0,0,22,267,0
5,Grande Saline,0,0,0,78,152,0
6,Gressier,0,0,0,5,113,0
7,Kenscoff,0,0,0,238,499,0
8,La Chapelle,0,0,0,0,66,0
9,Liancourt,0,0,0,38,76,0


In [10]:
#DATIMHIVSTAT.to_excel('./hivstat_results/DATIMHIVSTAT.xlsx',na_rep="",index=True)
DATIMHIVSTAT.to_excel(f"./hivstat_results/DATIMHIVSTAT_{date_du_jour}.xlsx",na_rep="",index=True)

In [11]:
HIVSTAT[HIVSTAT.positive=="positive"].id_patient

4858    125012
Name: id_patient, dtype: int64

# Remember to delete Johanne Faustin Information